# Inserting and Updating Objects
Demonstrate creation and updating of different objects with the Rhino Health Python SDK

#### Prerequisites 
1. Have projects called "Test" and "Test2" that you have created
2. Have a data schema that you can use for importing a dataset or use inferred data schema
3. Have files on your Rhino Client that can be used to import a dataset with the data schema
4. Have a container image pushed to your ECR repo with any code (just used for creating an object, not for running the code)

### Initialization and Login

In [ ]:
from getpass import getpass
import rhino_health as rh
from rhino_health.lib.endpoints.endpoint import NameFilterMode, VersionMode
from rhino_health.lib.endpoints.code_object.code_object_dataclass import CodeObjectCreateInput
from rhino_health.lib.endpoints.dataset.dataset_dataclass import DatasetCreateInput

In [ ]:
print("Logging In")
my_username = "my_email@example.com" # Replace this with the email you use to log into Rhino Health
my_workgroup_ecr_repo = "rhino-gc-workgroup-XXXXXXXXXXXXXX"  # Replace this with your workgroup's ECR repo
my_image_name = "upserting-objects"  # The name (tag) of the container image you pushed to your ECR repo
ecr_base_uri = rh.lib.constants.ECRService.PROD_URL
session = rh.login(username=my_username, password=getpass())
print("Logged In")

# Get Project By Name

In [ ]:
projects = session.project.search_for_projects_by_name("Test")
print("projects")
print([(x.name, x.uid) for x in projects][:10])

In [ ]:
project = session.project.get_project_by_name("Test2")
print(project)

In [ ]:
print(project.dict(include={'uid', 'name', 'description'}))

# Create Dataset

In [ ]:
data_schema_uid = "XXXXXXXX-XXXX-XXXX-XXXX-XXXXXXXXXXXX"  # Replace data schema UID, use empty string for inferred data schema
dataset_csv_location = "/rhino_data/test/dataset_data.csv"  # Replace dataset CSV location

In [ ]:
dataset_input = DatasetCreateInput(
            name="Upsert Dataset",
            description="Test Upsert Dataset",
            project_uid=project.uid,
            workgroup_uid=project.primary_workgroup_uid,
            data_schema_uid = data_schema_uid,
            csv_filesystem_location=dataset_csv_location,
            method="filesystem",
            is_data_deidentified=True,
        )
dataset = session.dataset.add_dataset(dataset_input)
print(dataset.uid)

In [ ]:
duplicate_dataset_input = session.dataset.add_dataset(dataset_input)
print(duplicate_dataset_input.uid)

In [ ]:
new_version_of_dataset = session.dataset.add_dataset(dataset_input, return_existing=False, add_version_if_exists=True)
print(new_version_of_dataset.uid, new_version_of_dataset.version)

# Get Dataset

In [ ]:
found_dataset = project.get_dataset_by_name("Upsert Dataset")
print(found_dataset.dict(include={'name', 'uid', 'version'}))

In [ ]:
older_dataset = project.get_dataset_by_name("Upsert Dataset", version=1)
print(older_dataset.dict(include={'name', 'uid', 'version'}))

In [ ]:
all_dataset_versions = project.search_for_datasets_by_name("upsert", version=VersionMode.ALL, name_filter_mode=NameFilterMode.CONTAINS)
print([dataset.dict(include={'name', 'uid', 'version'}) for dataset in all_dataset_versions])

In [ ]:
non_existent_dataset = project.get_dataset_by_name("I do not exist")
print (non_existent_dataset)

# Create CodeObject

In [ ]:
image_uri = f"{ecr_base_uri}/{my_workgroup_ecr_repo}:{my_image_name}"

test_code_object = CodeObjectCreateInput(
    name="Upsert Code Object",
    description="Test",
    input_data_schema_uids=[data_schema_uid],
    output_data_schema_uids=[data_schema_uid],
    project_uid=project.uid,
    code_type="Generalized Compute",
    config={"container_image_uri": image_uri},
)

code_object = session.code_object.create_code_object(
    test_code_object
)

print(code_object.uid)

In [ ]:
duplicate_code_object_input = session.code_object.create_code_object(test_code_object)
print(duplicate_code_object_input.uid)

In [ ]:
new_version_of_code_object = session.code_object.create_code_object(test_code_object, return_existing=False, add_version_if_exists=True)
print(new_version_of_code_object.dict(include={'name', 'uid', 'version'}))

# Get CodeObject

In [ ]:
found_code_object = project.get_code_object_by_name(code_object.name)
print(found_code_object.dict(include={'name', 'uid', 'version'}))

In [ ]:
older_code_object = project.get_code_object_by_name(code_object.name, version=0)
print(older_code_object.dict(include={'name', 'uid', 'version'}))

In [ ]:
all_code_object_versions = project.search_for_code_objects_by_name("upsert", version=VersionMode.ALL, name_filter_mode=NameFilterMode.CONTAINS)
print([code_object.dict(include={'name', 'uid', 'version'}) for code_object in all_code_object_versions])

In [ ]:
non_existent_code_object = project.get_code_object_by_name("I do not exist")
print (non_existent_code_object)

# Global Search

In [ ]:
partial_search = session.code_object.search_for_code_objects_by_name(
    "Test", version=1, name_filter_mode=NameFilterMode.CONTAINS
)
print("search 1")
print([(x.name, x.version) for x in partial_search][:10])

In [ ]:
partial_search = session.code_object.search_for_code_objects_by_name(
    "Upsert",
    version=VersionMode.LATEST,
    name_filter_mode=NameFilterMode.CONTAINS,
)
print("search latest")
print([(x.name, x.version) for x in partial_search])

In [ ]:
partial_search = session.code_object.search_for_code_objects_by_name(
    "Upsert",
    version=VersionMode.ALL,
    name_filter_mode=NameFilterMode.CONTAINS,
)
print("search latest")
print([(x.name, x.version) for x in partial_search])